<a href="https://colab.research.google.com/github/Yunkoo-GIT/Programming/blob/main/20210513_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pytorch

  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-sp539kjk/pytorch/setup.py'"'"'; __file__='"'"'/tmp/pip-install-sp539kjk/pytorch/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-kflfcm1f/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [2]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [20]:
#-*- coding: utf-8 -*-
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)
X = mnist.data
y = mnist.target

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

Xtrain, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=0)
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(list(map(int, y_train)))
y_test = torch.LongTensor(list(map(int, y_test)))

import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

X_train = X_train.view(-1,1,28,28).float()
X_test = X_test.view(-1,1,28,28).float()
print(X_train.shape)
print(X_test.shape)

train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)
BATCH_SIZE = 32
loader_train = DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
loader_test = DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1,32,kernel_size=5)
    self.conv2 = nn.Conv2d(32,32,kernel_size=5)
    self.conv3 = nn.Conv2d(32,64,kernel_size=5)
    self.fc1 = nn.Linear(3*3*64,256)
    self.fc2 = nn.Linear(256,10)

    self.loss_fn = nn.CrossEntropyLoss()
    self.optimizer = optim.Adam(self.parameters(), lr=0.01)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(F.max_pool2d(self.conv2(x),2))
    x = F.dropout(x, p=.05, training=self.training)
    x = F.relu(F.max_pool2d(self.conv3(x),2))
    x = F.dropout(x, p=.05, training=self.training)
    x = x.view(-1,3*3*64)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)

#torch.nn.Conv2d(in_channels, out_channels, kernel_size,
#                stride=1, padding=0, dilation=1, groups=1,
#                bias=True, padding_mode='zeros')

def fit(model, loader_train):
  optimizer = torch.optim.Adam(model.parameters())
  error = nn.CrossEntropyLoss()
  EPOCHS = 1
  model.train()
  for epoch in range(EPOCHS):
    correct = 0
    for batch_idx, (X_batch, y_batch) in enumerate(loader_train):
      var_X_batch = Variable(X_batch).float()
      var_y_batch = Variable(y_batch)
      optimizer.zero_grad()
      output = model(var_X_batch)
      loss = error(output, var_y_batch)
      loss.backward()
      optimizer.step()
      predicted = torch.max(output.data, 1)[1]
      correct += (predicted == var_y_batch).sum()
      if batch_idx % 50 == 0:
        print('에포크: {} [{}/{} ({:.0f}%)]\t 손실함수: {:.6f}\t Accuracy:{:.3f}%'.format(
            epoch, batch_idx*len(X_batch), len(loader_train),
            100.*batch_idx / len(loader_train), loss.data,
            correct*100. / (BATCH_SIZE*(batch_idx+1))))
        
def evaluate(model):
  correct = 0
  for test_imgs, test_labels in loader_test:
    test_imgs = Variable(test_imgs).float()
    output = model(test_imgs)
    predicted = torch.max(output,1)[1]
    correct += (predicted == test_labels).sum()
  print("테스트 데이터 정확도: {:.3f}% ".format(float(correct) /
        (len(loader_test)*BATCH_SIZE)))
cnn = CNN()
evaluate(cnn)
fit(cnn, loader_train)
cnn.eval()  #모델 테스트 모드로 전환
evaluate(cnn)
index = 10  #테스트 데이터 중에서 확인해볼 데이터의 인덱스
data = X_test[index].view(-1,1,28,28).float()
output = cnn(data)  #모델 적용
print('{} 번째 학습데이터의 테스트 결과: {}'.format(index, output))
_, predicted = torch.max(output, 1)
print('{} 번째 데이터의 예측: {}'.format(index, predicted.numpy()))
print('실제 레이블: {}'.format(y_test[index]))

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])
테스트 데이터 정확도: 0.091% 
에포크: 0 [0/1875 (0%)]	 손실함수: 2.304294	 Accuracy:9.375%
에포크: 0 [1600/1875 (3%)]	 손실함수: 0.785772	 Accuracy:47.549%
에포크: 0 [3200/1875 (5%)]	 손실함수: 0.614308	 Accuracy:65.377%
에포크: 0 [4800/1875 (8%)]	 손실함수: 0.294450	 Accuracy:73.406%
에포크: 0 [6400/1875 (11%)]	 손실함수: 0.375716	 Accuracy:77.659%
에포크: 0 [8000/1875 (13%)]	 손실함수: 0.253167	 Accuracy:80.627%
에포크: 0 [9600/1875 (16%)]	 손실함수: 0.176691	 Accuracy:82.859%
에포크: 0 [11200/1875 (19%)]	 손실함수: 0.105122	 Accuracy:84.633%
에포크: 0 [12800/1875 (21%)]	 손실함수: 0.085106	 Accuracy:85.934%
에포크: 0 [14400/1875 (24%)]	 손실함수: 0.095087	 Accuracy:87.050%
에포크: 0 [16000/1875 (27%)]	 손실함수: 0.011589	 Accuracy:87.849%
에포크: 0 [17600/1875 (29%)]	 손실함수: 0.412019	 Accuracy:88.481%
에포크: 0 [19200/1875 (32%)]	 손실함수: 0.195625	 Accuracy:89.117%
에포크: 0 [20800/1875 (35%)]	 손실함수: 0.091461	 Accuracy:89.636%
에포크: 0 [22400/1875 (37%)]	 손실함수: 0.014739	 Accuracy:90.139%
에포크: 0 [24000/1875 (40%)]	 손실함수: